# The Business Problem

**Descrition of problem:**

The client is a credit card company and have brought a dataset that includes some demoegraphics and recent finance data, over the past 6 months, foa a sample of 30,000 of their account holders. This data is at the credit account level. The rowns are labeled by wheter, in the next month after the 6-monts historical data period, an account owner has defaulted, or in other, words, failed to make the mininum payment.

**Goal:**

Your goal is to develop a predictive model for whether an account will default next month, given demographics and historical data.

In [3]:
import pandas as pd
df = pd.read_excel('default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

A review of data set information (Data dictionary)

- LIMIT_BAL: Amount of credit provided including individual consumer and family credit
- SEX: Gender(1 = male; 2 = female)
- EDUCATION: Education (1 = graduation school; 2 = university; 3 = high scholl; 4 = others)
- MARRIAGE: Marital status ( 1 = married; 2 = single; 3 = others)
- AGE: Age (year)
- PAY_1-PAY_6: A record of past payments. Past monthly payments, recorded from April to September, are stored in theses columns, PAY_1: Represents the rapyment status in September, and the others for before mothns
- BILL_AMT1-BILL_AMT6: BILL statement amount (in NT dollar),BILL_AMT1, represents the bill statement amount in September, and the others for before months
- PAY_AMT1-PAY_AMT6: Amount of previous payment (NT dollar), PAY_AMT1, represents the amount paid in September, and the other for before months.

## Verifying Basic Data Integrity

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


The total unique values is

In [6]:
df['ID'].nunique()

29687

and the total of rows and columns are 

In [7]:
df.shape

(30000, 25)

In [8]:
30000-29687

313

The first point is that the number of rows is large than the unique values `ID`, this can be indicated that there is some duplication of IDs. 

We can start if the question. How munch? Is one ID duplicated many times or how many IDs are duplicated?
To answer theses quesiotns we can counts.

In [9]:
id_counts = df['ID'].value_counts()
id_counts.head() # Shows only the first five rows by default

ID
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
75938fec-e5ec    2
2a793ecf-05c6    2
Name: count, dtype: int64

In [10]:
id_counts.value_counts()

count
1    29374
2      313
Name: count, dtype: int64

We can see now that mosts IDs accur exactly once, but 313 IDs apear twice. Now we can anwser part of our questions. First we know how many IDs are duplicated and no ID occurs more than twice.

In [11]:
dupe_mask = id_counts == 2  # Geting the ID which are duplicated
dupe_mask[0:5]

ID
89f8f447-fca8    True
7c9b7473-cc2f    True
90330d02-82d9    True
75938fec-e5ec    True
2a793ecf-05c6    True
Name: count, dtype: bool

In [12]:
id_counts.index[0:5] # And your index

Index(['89f8f447-fca8', '7c9b7473-cc2f', '90330d02-82d9', '75938fec-e5ec',
       '2a793ecf-05c6'],
      dtype='object', name='ID')

In [13]:
dupe_ids = id_counts.index[dupe_mask] # Save the index for duplicate ID

In [14]:
dupe_ids = list(dupe_ids) # Convert dupe_ids to a list
len(dupe_ids)

313

The len of dupe_ids agree with number of duplicate IDs.
Now we can move on to see if something can be different between these duplicate entries. 

In [15]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
5033,89f8f447-fca8,320000,2,2,1,32,0,0,0,0,...,169371,172868,150827,8000,8000,5500,6100,6000,5000,0
5133,89f8f447-fca8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15879,7c9b7473-cc2f,90000,2,1,1,29,0,0,0,0,...,27751,20292,14937,2967,2007,1429,1092,412,263,0
15979,7c9b7473-cc2f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29646,90330d02-82d9,70000,1,2,1,29,0,0,0,0,...,10694,27908,11192,2009,1404,3016,20001,2000,5002,0
29746,90330d02-82d9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can see that one rows has the information and the other only zero value.
Now we can create a Boolean matrix of the same size as the the entire DataFrame

In [16]:
df_zero_mask = df == 0

In [17]:
df_zero_mask

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,False,True,True,True,True,False
1,False,False,False,False,False,False,False,False,True,True,...,False,False,False,True,False,False,False,True,False,False
2,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,False,False,False,False,False,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,True
29996,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,True,True
29997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
29998,False,False,False,False,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False


We want to create a DataFrame with Boolean Series that telling us which rows have all zero besides the ID

In [18]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)

In [19]:
sum(feature_zero_mask)

315

The sum telling us that we have 315 rows with all zero, whihc is greater than the number of duplicate IDs (313). So if we delete all zero rows we can resolve our problem.

In [20]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy() # the ~ is a lofical not operator, to select all the rows that don't have zeros for all the features

In [21]:
df_clean_1.shape

(29685, 25)

In [22]:
df_clean_1['ID'].nunique()

29685

After these proceding we have the succesfully elimination of all duplicates IDs. 

In [23]:
df_clean_1.to_csv('df_clean_1.csv', index=False)

## Exploring and Cleaning the Data

In [24]:
df_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          29685 non-null  object
 1   LIMIT_BAL                   29685 non-null  int64 
 2   SEX                         29685 non-null  int64 
 3   EDUCATION                   29685 non-null  int64 
 4   MARRIAGE                    29685 non-null  int64 
 5   AGE                         29685 non-null  int64 
 6   PAY_1                       29685 non-null  object
 7   PAY_2                       29685 non-null  int64 
 8   PAY_3                       29685 non-null  int64 
 9   PAY_4                       29685 non-null  int64 
 10  PAY_5                       29685 non-null  int64 
 11  PAY_6                       29685 non-null  int64 
 12  BILL_AMT1                   29685 non-null  int64 
 13  BILL_AMT2                   29685 non-null  int64 


Look the information from df_clean_1, it is possible to see that all rows don't have `non-null`, and the only columns that don't have int64 are `ID` and `PAY_1`. For `ID` we now that them contains strings. But what about `PAY_1`?

In [25]:
df_clean_1['PAY_1'].head(5)

0     2
1    -1
2     0
3     0
4    -1
Name: PAY_1, dtype: object

The problem with thise data is the fact the `PAY_1` columns need to shown the payment status of the monst recent month'bill, using the values -1,1,2 ..., but we have a 0 value, which is not documented in the data dictionary.

*-1 = pay duly; 1 = payment delay for one month; ...*

Looking the value counts for the `PAY_1`, we get

In [26]:
df_clean_1['PAY_1'].value_counts()

PAY_1
0                13087
-1                5047
1                 3261
Not available     3021
-2                2476
2                 2378
3                  292
4                   63
5                   23
8                   17
6                   11
7                    9
Name: count, dtype: int64

The proceding output reveals the presence of two undocumented values: 0 and -2 the 'Not available' string. 

In [28]:
valid_pay_1_mask = df_clean_1['PAY_1'] != 'Not available'  # != does not equal, here we find the pay which is only available
valid_pay_1_mask[0:5]

0    True
1    True
2    True
3    True
4    True
Name: PAY_1, dtype: bool

In [29]:
sum(valid_pay_1_mask)

26664

Now we can clean the data by eliminating the rows with the missing values of `PAY_1` as shown:

In [30]:
df_clean_2 = df_clean_1.loc[valid_pay_1_mask,:].copy()

In [31]:
df_clean_2.shape

(26664, 25)

In [32]:
df_clean_2['PAY_1'].value_counts()

PAY_1
0     13087
-1     5047
1      3261
-2     2476
2      2378
3       292
4        63
5        23
8        17
6        11
7         9
Name: count, dtype: int64

Now all data from `PAY_1` can be describe as int64 type using the `.astype` method.

In [34]:
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')
df_clean_2[['PAY_1', 'PAY_2']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 26664 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   PAY_1   26664 non-null  int64
 1   PAY_2   26664 non-null  int64
dtypes: int64(2)
memory usage: 624.9 KB


The business partner send the following information:

- -2 means the account started that month with a zero balance and never used any credit
- -1 means the account had a balance that was paid in full
- 0 means that at least the minimum paymennt was made, but the entire balance wasn't paid.

In [35]:
df_clean_2.to_csv('df_clean_2.csv', index = False)